In [ ]:
import pandas as pd
import csv
import json

# Extract abstract documents from .tok file

In [ ]:
# Read the .tok file and extract the original text
tok_file_path = '/content/eng.dep.covdtb_test.tok'
doc_id = None
text_lines = []
df = pd.DataFrame(columns=['Document ID', 'Document Abstract'])

with open(tok_file_path, 'r', encoding='utf-8') as tok_file:
  for line in tok_file:
    if line.startswith("# newdoc_id"):
      doc_id = line.split('=')[1].strip()
    elif line and line[0].isdigit():
      parts = line.strip().split('\t')
      if len(parts) >= 2:
          token = parts[1]
          text_lines.append(token)
    elif line:
      original_text = ' '.join(text_lines)
      df.loc[len(df)] = [doc_id, original_text]
      text_lines = []
  original_text = ' '.join(text_lines)
  df.loc[len(df)] = [doc_id, original_text]

df.head()

,Document ID,Document Abstract
0,00c862a6402381a8541bcffe843696e059095300,"In this study , we investigated turkey reoviru..."
1,0146a0b36f80ebcb868c7e16fb60c1a9d2afbf83,This is an extensive review on epiphytic plant...
2,0232f18e7bbf3192e9e34da1c0abe2a20b2ba1e4,"Herein , the LASSBio Chemical Library is prese..."
3,09bdbea1700a3826acc33484ed51e2fa8c23f047,Data - centric models of COVID-19 have been tr...
4,0b362bc7e9278b56c2740ab7bc10e91c036f835a,We study seasonal epidemic spreading in a susc...


In [ ]:
df.iloc[0, 1]

'In this study , we investigated turkey reovirus ( TReoV ) in tissue samples from young birds , aged 15 days . RT - PCR for TReoV detected 3.3 % positive samples and TReoV was successfully isolated in Vero cells . Histological analysis of positive bursa of Fabricius ( BF ) revealed atrophied follicles and lymphocyte depletion . The number of CD8 ? , CD4 ? and IgM ? cells was lower in infected BF . Phylogenetic analysis based on S3 gene showed that the Brazilian TReoV isolates clustered in a single group with 98 - 100 % similarity to TReoV strains circulating in the United States . This is the first indication that TReoV infection may be a contributing factor to immunosuppression in young birds .'

In [ ]:
df.shape

(150, 2)

In [ ]:
# Create a CSV file and write the data
csv_file_path = 'eng_dep_covdtb_texts_test.csv'
df.to_csv(csv_file_path, index=False)

# Extract EDUs from each document abstract

In [ ]:
data_with_ids = pd.read_csv('/content/drive/MyDrive/chat_gpt_datasets/eng_dep_covdtb_texts_test.csv')
document_ids = data_with_ids['Document ID']
document_ids.shape

(150,)

In [ ]:
edu_df = pd.DataFrame(columns=['Document ID', 'Document EDUs'])

for id in document_ids:
  edu_file_path = '/content/drive/MyDrive/chat_gpt_datasets/test/' + str(id) + '.dep'
  with open(edu_file_path, 'r', encoding='utf-8') as edu_file:
    data = json.load(edu_file)
    root_data = data["root"]
    edu_list = []
    tmp = 1

    for item in root_data:
        id_value = item["id"]
        text_value = item["text"]
        if id_value == 0:
          continue
        if tmp == id_value:
          edu_list.append(str(text_value))
          tmp += 1
        else:
          print("Id value: " + str(id_value) + ",   tmp value: " + str(tmp))

    edu_df.loc[len(edu_df)] = [id, edu_list]

edu_df.head()

,Document ID,Document EDUs
0,00c862a6402381a8541bcffe843696e059095300,"[In this study , we investigated turkey reovir..."
1,0146a0b36f80ebcb868c7e16fb60c1a9d2afbf83,[This is an extensive review on epiphytic plan...
2,0232f18e7bbf3192e9e34da1c0abe2a20b2ba1e4,"[Herein , the LASSBio Chemical Library is pres..."
3,09bdbea1700a3826acc33484ed51e2fa8c23f047,[Data - centric models of COVID-19 have been t...
4,0b362bc7e9278b56c2740ab7bc10e91c036f835a,[We study seasonal epidemic spreading in a sus...


In [ ]:
edu_df.shape

(150, 2)

In [ ]:
# Create a CSV file and write the data
csv_file_path = 'eng_dep_covdtb_edus_test.csv'
edu_df.to_csv(csv_file_path, index=False)

# Extract discourse relations from the texts

In [ ]:
data_with_relations = pd.read_csv('/content/drive/MyDrive/chat_gpt_datasets/eng.dep.covdtb_test.rels.csv', sep='\t')
data_with_relations.tail(10)

,doc,unit1_toks,unit2_toks,unit1_txt,unit2_txt,s1_toks,s2_toks,unit1_sent,unit2_sent,dir,orig_label,label
2576,fcc503a22ab18a67222842a84aa2ba08e0e4865e,70-88,89-97,Hence in every stage of dissection room activi...,so as to set a standard for the students,70-97,70-97,Hence in every stage of dissection room activi...,Hence in every stage of dissection room activi...,1<2,enablement,enablement
2577,fcc503a22ab18a67222842a84aa2ba08e0e4865e,98-103,104-107,The present review is an attempt,to collate the recommendations,98-121,98-121,The present review is an attempt to collate th...,The present review is an attempt to collate th...,1<2,enablement,enablement
2578,fcc503a22ab18a67222842a84aa2ba08e0e4865e,104-107,108-121,to collate the recommendations,documented by researchers as per ethical guide...,98-121,98-121,The present review is an attempt to collate th...,The present review is an attempt to collate th...,1<2,elaboration,elaboration
2579,fcc503a22ab18a67222842a84aa2ba08e0e4865e,98-103,122-128,The present review is an attempt,The review highlights on the ethical norms,98-121,122-149,The present review is an attempt to collate th...,The review highlights on the ethical norms whi...,1<2,elaboration,elaboration
2580,fcc503a22ab18a67222842a84aa2ba08e0e4865e,122-128,129-134,The review highlights on the ethical norms,which needs to be adhered to,122-149,122-149,The review highlights on the ethical norms whi...,The review highlights on the ethical norms whi...,1<2,elaboration,elaboration
2581,fcc503a22ab18a67222842a84aa2ba08e0e4865e,129-134,135-149,which needs to be adhered to,while receiving the human body of a donor and ...,122-149,122-149,The review highlights on the ethical norms whi...,The review highlights on the ethical norms whi...,1<2,temporal,temporal
2582,fcc503a22ab18a67222842a84aa2ba08e0e4865e,122-128,150-164,The review highlights on the ethical norms,It reflects on ideal ethical behaviour in the ...,122-149,150-180,The review highlights on the ethical norms whi...,It reflects on ideal ethical behaviour in the ...,1<2,elaboration,elaboration
2583,fcc503a22ab18a67222842a84aa2ba08e0e4865e,150-164,165-180,It reflects on ideal ethical behaviour in the ...,and finally emphasize on the respectful dispos...,150-180,150-180,It reflects on ideal ethical behaviour in the ...,It reflects on ideal ethical behaviour in the ...,1<2,elaboration,elaboration
2584,fcc503a22ab18a67222842a84aa2ba08e0e4865e,98-103,181-200,The present review is an attempt,The intended purpose of this article is to sup...,98-121,181-205,The present review is an attempt to collate th...,The intended purpose of this article is to sup...,1<2,enablement,enablement
2585,fcc503a22ab18a67222842a84aa2ba08e0e4865e,181-200,201-205,The intended purpose of this article is to sup...,used in anatomical dissection .,181-205,181-205,The intended purpose of this article is to sup...,The intended purpose of this article is to sup...,1<2,elaboration,elaboration


In [ ]:
len(data_with_relations['doc'].unique())

150

In [ ]:
df_rel = pd.DataFrame()
df_rel['Document ID'] = data_with_relations['doc']
df_rel['unit1_txt'] = data_with_relations['unit1_txt']
df_rel['unit2_txt'] = data_with_relations['unit2_txt']
df_rel['label'] = data_with_relations['label']
df_rel.head(5)

,Document ID,unit1_txt,unit2_txt,label
0,00c862a6402381a8541bcffe843696e059095300,"In this study , we investigated turkey reoviru...",aged 15 days .,elaboration
1,00c862a6402381a8541bcffe843696e059095300,"In this study , we investigated turkey reoviru...",RT - PCR for TReoV detected 3.3 % positive sam...,findings
2,00c862a6402381a8541bcffe843696e059095300,RT - PCR for TReoV detected 3.3 % positive sam...,and TReoV was successfully isolated in Vero ce...,joint
3,00c862a6402381a8541bcffe843696e059095300,RT - PCR for TReoV detected 3.3 % positive sam...,Histological analysis of positive bursa of Fab...,elaboration
4,00c862a6402381a8541bcffe843696e059095300,Histological analysis of positive bursa of Fab...,"The number of CD8 ? , CD4 ? and IgM ? cells wa...",elaboration


In [ ]:
# Create a CSV file and write the data
csv_file_path = 'eng.dep.covdtb_test_relations.csv'
df_rel.to_csv(csv_file_path, index=False)